In [ ]:
import numpy as np

In [92]:
def sampleOne(driver, driver_tripList, K = 200):
    """
    :param: driver: int, the ID if of the driver you want to simulate fake trips for
    :param: driver_tripList: list, list of tuples of unique (driver, trip)
    :param: K: int, number of fake trips you want to create
    :output: faketrips: list, tuple of tuples with ((real_driverID, real_tripID),(fake_driver, fake_trip))
    """
    candidates = [x for x in driver_tripList if x[0] != driver]
    samples = np.random.choice(np.arange(0,len(candidates)), K, True)
    sampleTups = [candidates[x] for x in samples]
    IDDriv = [driver] * K
    IDTrip = np.arange(1,K+1) + K
    sampleIDs = zip(IDDriv, IDTrip)
    faketrips = zip(sampleTups, sampleIDs)
    return faketrips

In [ ]:
def labelOne(rdd, driver, K = 200):
    """
    :paraasldkjfljkadsf
    """
    driver_tripList = set(rdd.map(lambda line: line[0]).collect)
    samples = sampleOne(driver, driver_tripList, K = K)
    rddfilter = rdd.filter(lambda x: x[0] in zip(*samples)[0])
    rddsamples = sc.parallelize(samples)
    rddnews = rddfilter.join(rddsamples)
    rddzeros = rddnews.map(lambda x: (x[1][1],x[1][0],0.0))
    rddones = rdd.filter(lambda x: x[0][0] == driver)
    rddones = rddones.map(lambda x: (x[0],x[1],1))
    finalrdd = rddones.union(rddzeros)
    return finalrdd

In [94]:
test.map(lambda x:x[0]).collect()

[(1, 1), (1, 1), (1, 1), (2, 1), (2, 1), (2, 2)]

In [99]:
sampleOne(2, test.map(lambda x:x[0]).collect())

[((1, 2), (2, 201)),
 ((1, 1), (2, 202)),
 ((1, 1), (2, 203)),
 ((1, 1), (2, 204)),
 ((1, 1), (2, 205)),
 ((1, 1), (2, 206)),
 ((1, 1), (2, 207)),
 ((1, 1), (2, 208)),
 ((1, 1), (2, 209)),
 ((1, 2), (2, 210)),
 ((1, 1), (2, 211)),
 ((1, 1), (2, 212)),
 ((1, 2), (2, 213)),
 ((1, 2), (2, 214)),
 ((1, 2), (2, 215)),
 ((1, 1), (2, 216)),
 ((1, 1), (2, 217)),
 ((1, 1), (2, 218)),
 ((1, 2), (2, 219)),
 ((1, 1), (2, 220)),
 ((1, 2), (2, 221)),
 ((1, 1), (2, 222)),
 ((1, 1), (2, 223)),
 ((1, 2), (2, 224)),
 ((1, 1), (2, 225)),
 ((1, 1), (2, 226)),
 ((1, 2), (2, 227)),
 ((1, 1), (2, 228)),
 ((1, 1), (2, 229)),
 ((1, 1), (2, 230)),
 ((1, 1), (2, 231)),
 ((1, 2), (2, 232)),
 ((1, 1), (2, 233)),
 ((1, 2), (2, 234)),
 ((1, 2), (2, 235)),
 ((1, 1), (2, 236)),
 ((1, 1), (2, 237)),
 ((1, 1), (2, 238)),
 ((1, 1), (2, 239)),
 ((1, 1), (2, 240)),
 ((1, 2), (2, 241)),
 ((1, 1), (2, 242)),
 ((1, 1), (2, 243)),
 ((1, 1), (2, 244)),
 ((1, 1), (2, 245)),
 ((1, 1), (2, 246)),
 ((1, 1), (2, 247)),
 ((1, 1), (2,

In [97]:
test = sc.parallelize([((1,1),(0,0)),((1,1),(2,1)),((1,2),(4,1)),((2,1),(5,2)),((2,1),(8,1)),((2,2),(6,5))])

In [98]:
test.collect()

[((1, 1), (0, 0)),
 ((1, 1), (2, 1)),
 ((1, 2), (4, 1)),
 ((2, 1), (5, 2)),
 ((2, 1), (8, 1)),
 ((2, 2), (6, 5))]

In [61]:
# create fake lists for driver, returns list of tuples ((real driver, real trip),(driver being simulated, trip being simulated))
def sampleOne(driver, driver_list, trip_list, K = 200):
    otherDriv = [x for x in driver_list if x != driver]
    randDriv= np.random.choice(otherDriv, K, True)
    randTrip = np.random.choice(trip_list, K, True)
    IDDriv = [driver] * K
    IDTrip = np.arange(1,K+1) + K
    sampleIDs = zip(IDDriv, IDTrip)
    sample = zip(randDriv, randTrip)
    result = zip(sample, sampleIDs)
    return list

In [ ]:
def labelMethod1(x):
    drivers = x.map(lambda line: line[0][0]).collect()
    trips = x.map(lambda line: line[0][1]).collect()
    totSamp = []
    for i in drivers: 
        otherDriv = [x for x in drivers if x != i]
        randDriv= np.random.choice(otherDriv, 200, True)
        randTrip = np.random.choice(trips, 200, True)
        IDDriv = [i] * 200
        IDTrip = np.arange(1,201) + 200
        sampleIDs = zip(IDDriv, IDTrip)
        sample = zip(randDriv, randTrip)
        result = zip(sample, sampleIDs)
        
        
        

In [4]:
test.sample(False,.3)

PythonRDD[2] at RDD at PythonRDD.scala:43

In [6]:
K = [1,2,4]

In [26]:
[1]*5

[1, 1, 1, 1, 1]

In [34]:
np.arange(1,201) + 200

array([201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213,
       214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226,
       227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239,
       240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252,
       253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265,
       266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278,
       279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291,
       292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304,
       305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317,
       318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330,
       331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343,
       344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356,
       357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369,
       370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 38

In [9]:
[i for i in K]

[1, 2, 4]

In [20]:
np.random.choice(K,10, True)

array([2, 2, 2, 1, 1, 4, 4, 4, 1, 4])

In [36]:
ONE = zip(np.random.choice(K,10, True),np.random.choice(K,10, True)) + zip(np.random.choice(K,10, True),np.random.choice(K,10, True))

In [22]:
K + K

[1, 2, 4, 1, 2, 4]

In [38]:
J = ['a','b','c']

In [42]:
OOO = zip(np.random.choice(K,10, True), np.random.choice(K,10, True))

In [43]:
KKK = zip(np.random.choice(J,10, True), np.random.choice(J,10, True))

In [44]:
OOO

[(4, 2),
 (4, 1),
 (2, 4),
 (2, 4),
 (1, 1),
 (2, 4),
 (1, 1),
 (4, 2),
 (1, 2),
 (1, 2)]

In [46]:
KKK

[('a', 'a'),
 ('c', 'b'),
 ('c', 'c'),
 ('a', 'a'),
 ('a', 'b'),
 ('b', 'b'),
 ('c', 'c'),
 ('b', 'b'),
 ('a', 'a'),
 ('c', 'c')]

In [49]:
TTT = zip(OOO, KKK)

In [52]:
(4,2) in zip(*TTT)[0]

True

In [54]:
check = sc.parallelize(TTT)

In [58]:
check.first()

((4, 2), ('a', 'a'))